# Sentiment Analysis

Dans ce TP, on va créer et entraîner un réseau de neuronnes récurrent pour l'analyse de sentiment. 

Le jeu de données que l'on utilise est *IMDB Movie Reviews* où on trouve des revues de films avec chacune sa *polarité* (positif/negatif).

Le TP est composé comme suit

1.   Chargement et visualisation des données
2.   Conception de réseau
3.   Entraînement de réseau
4.   Validation de réseau

Pour gagner du temps, on vous fournit un squelette de code. Les codes ne sont pas complets, et il vous sera demandé de les compléter certains endroits pour qu'ils fonctionnent.

Lorsque un bout de code est à compléter, il sera marqué comme ceci
```python
def factorial(n):
  if n == 0:
    return 1
  ### VOTRE CODE COMMENCE ICI ###
  ### VOTRE CODE TERMINE ICI  ###

assert factorial(5) == 120
```
où `assert factorial(5) == 120` sert à tester votre code.

Vous devriez le compléter comme ceci
```python
def factorial(n):
  if n == 0:
    return 1
  ### VOTRE CODE COMMENCE ICI ###
  return n * factorial(n - 1)
  ### VOTRE CODE TERMINE ICI  ###
```


Vous n'êtes **pas** obligé de maîtriser les autres codes.

Enfin, il est recommendé de compléter le TP dans cet ordre. Toutefois, si vous sentir bloqué, n'hésitez pas à nous soliciter.

On commence par importer des librairies dont on aura besoin par la suite. Vous pouvez aussi importer d'autres librairies de vos choix.

Pas de panique si vous n'êtes pas familier avec tous les imports, on vous expliquera plus tard dans le TP.

In [0]:
import os
import random
from glob import iglob
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image
from keras.datasets import imdb
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM
from keras.utils import plot_model

## 1. Chargement et visualisation des données

Le jeu de données *IMDB Movie reviews* est composé de 50000 revues de film issues du site IMDB avec chacune sa polarité (i.e. sentiment positif ou sentiment négatif).

Il est séparé en deux sous-ensemble de données : 

*   train: 25000 exemples de données d'entraînement, qui nous servent à **entraîner** les modèles
*   test: 25000 exemples de données de teste, qui seront utilisé pour **évaluer** les modèles

On télécharge d'abord les données brutes (ce qui prendra une dizaine de secondes) avant de les extraire dans un dossier.

In [0]:
! wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz && tar xf aclImdb_v1.tar.gz

Voici un exemple d'une revue positive

In [0]:
! cat aclImdb/train/pos/0_9.txt

et voici un exemple d'une revue négative

In [0]:
! cat aclImdb/train/neg/0_3.txt

On vous fournit une fonction pour charger les données d'entraînement et celles de test en mémoire.



In [0]:
def load_data(path, shuffle=True, seed=42):
  def load(train_or_test):
    data = []
    for polarity in ['pos', 'neg']:
      for filename in iglob(os.path.join(path, train_or_test, polarity, '*.txt')):
        with open(filename, 'r') as f:
          example = f.readline().strip()
          target = 1 if polarity == 'pos' else 0
          data.append((example, target))
    if shuffle:
      random.seed(seed)
      random.shuffle(data)
    X = [pair[0] for pair in data]
    y = [pair[1] for pair in data]
    return (X, y)
  return load('train'), load('test')

In [0]:
(X_train_text, y_train), (X_test_text, y_test) = load_data('aclImdb/')

`X_train_text` et `X_test_text` sont des revues, alors que `y_train` et `y_test` sont leur polarité (`0` pour négatif et `1` pour positif).

On vérifie que les données sont bien chargées.

In [0]:
assert len(X_train_text) == 25000
assert len(y_train) == 25000
assert len(X_test_text) == 25000
assert len(y_test) == 25000

On peut examiner aléatoirement une revue et sa polarité. (N'hésitez pas à exécuter la cellule plusieurs fois, vous aurez des résultats différents).

In [0]:
idx = random.randint(0, len(X_train_text))
print(f"Revue:\t\t{X_train_text[idx]}")
print(f"Polarité:\t{y_train[idx]}")

### 1.1 Tokenisation

En traitement du langage naturel, on commence (quasiment) toujours par tokeniser les données. Ce qu'un *token* représente dépend de la granularité qu'on cherche. Dans notre TP, on s'intéresse aux mots (*word-level*) plutôt qu'aux caractères (*character-level*).

Concrètement, on cherche à construire un *vocabulaire* comme ceci
```
<UNK>: 1
hello: 2
world: 3
```
qui associe aux mots un indice entier. Comme  convention, on ajoute un mot fictif `<UNK>` dans notre vocabulaire qui sert à representer tous les mots inconnus.

Avec le vocabulaire (de trois mots seulement!!!) ci-dessus, la phrase 
```
my hello word
```
sera transformée en un vecteur d'entiers
```
[1, 2, 3]
```
On rappelle que le mot `my` est inconnu pour ce vocabulaire, et c'est pourquoi il devient `1` (l'indice de `<UNK>`).

Etant donné un corpus (dans notre cas, le jeu de données IMDB), la taille du vocabulaire est un paramètre à choisir. 

Si on prend une taille trop petite, on aura des `<UNK>` partout, et il est peu propable que notre modèle apprenne quelque chose. 

Par contre, si on prend une taille trop grande, notre modèle s'appuyera peut-être sur des mots très rares ce qui n'est pas forcément une bonne chose.


En pratique, on prend les $n$ mots les plus fréquents dans le corpus. Et dans le cadre de ce TP, on prend (de façon arbitraire) $n = 6000$. Un bon exercice sera de modifer cette valeur et de regarder l'effet.

In [0]:
num_words = 6000

Keras fournit beaucoup d'utilitaires pour pré-processer les textes. En particulier, il existe une classe [Tokenizer](https://keras.io/preprocessing/text/) qui fait toute la tokenisation!

On initialise un tokeniser en lui indiquant la taille du vocabulaire et le token pour représenter les mots inconnus (*out of vocabulary (oov)*). Ensuite on *fit* le tokeniser sur notre jeu d'entraînement.

> A ne pas faire : fit le tokeniser sur le jeu complet (l'union de jeu d'entraînement et le jeu de teste). Pourquoi ?

In [0]:
tokenizer = Tokenizer(num_words=num_words, oov_token='<UNK>')
tokenizer.fit_on_texts(X_train_text)

A l'aide de ce tokeniser, on va pouvoir transformer notre jeux en des vecteurs.

A vous de coder !

> Indice : utiliser cette fonction [ceci](https://stackoverflow.com/a/55294888)

In [0]:
# X_train et X_test sont une liste de liste
X_train = ### VOTRE CODE ICI ###
X_test = ### VOTRE CODE ICI ###

In [0]:
assert len(X_train) == 25000
assert len(X_test) == 25000

On peut comparer le corpus avant et après tokenisation (n'hésitez pas à exécuter plusieurs fois la cellule)

In [0]:
idx = random.randint(0, len(X_train_text))

print(f"Donnée brute : {X_train_text[idx]}\n")
print(f"Après tokenisation : {X_train[idx]}\n")
print(f"  Avec des mots: {tokenizer.sequences_to_texts([X_train[idx]])[0]}")

### 1.2 Zero Padding

Un dernier détail technique : pour pouvoir entraîner notre réseau en *batch*, il faut que tous les vecteurs dans `X_train` et `X_test` aient la *même longueur*. Pour ce faire, on utilise ce qu'on appelle le "zero padding". 

Concrètement, on se donne une longueur fixe `maxlen`, qui est elle aussi un paramètre à choisir. Si un vecteur dépasse cette longueur, on coupe et ignore le surplus ; si un vecteur est trop court, on le complète avec un token spécial (souvent 0, d'où le nom zero padding) jusqu'à cette longeur.

A titre d'exemple, supposons que `maxlen = 4`, alors

*   [1, 2, 3, 4, 5] deviendra [1, 2, 3, 4]
*   [1, 2] deviendra [1, 2, 0, 0]

Pour le reste du TP, on prendra `maxlen = 128`

In [0]:
maxlen = 128

Maintenant, on va appliquer le zero padding sur `X_train` et `X_test`.

A vous de coder !

> Indice : utiliser [cette fonction](https://keras.io/preprocessing/sequence/#pad_sequences)

In [0]:
X_train_padded = ### VOTRE CODE ICI ###
X_test_padded = ### VOTRE CODE ICI ###

In [0]:
assert X_train_padded.shape == (25000, maxlen)
assert X_test_padded.shape == (25000, maxlen)

## Conception du réseau récurrent

Il est enfin temps de construire notre réseau ! Pour attaquer ce type de problème, il est courant d'utiliser un réseau de neurones récurrent (RNN). En pratique, on utilise très rarement le RNN de base à cause du problème de [vanishing gradient](https://en.wikipedia.org/wiki/Vanishing_gradient_problem). 

Dans ce TP, on utilise une variante qui s'appelle LSTM (Long-Short Term Memory).

Notre réseau ressemble au schéma suivant

![Texte alternatif…](https://cdn-images-1.medium.com/max/1000/1*SICYykT7ybua1gVJDNlajw.png)

En Keras, la construction d'un tel réseau est très simple : il suffit de définir un modèle séquentiel et d'y ajouter couche par couche les blocs suivants

1.   [Embedding](https://keras.io/layers/embeddings/) avec `output_dim = 128`. Fixez la valeur de `input_dim`, et laissez les autres paramètres par défaut.
2.   [LSTM](https://keras.io/layers/recurrent/#lstm), avec `units = 32`. Laissez les autres paramètres par défaut.
3.   [Dense](https://keras.io/layers/core/), avec `units = 1` et `activation = 'sigmoid'`. Laissez les autres paramètres par défaut.


In [0]:
model = Sequential()
model.add(### VOTRE CODE ICI ###)
model.add(### VOTRE CODE ICI ###)
model.add(### VOTRE CODE ICI ###)

En Keras, un modèle doit être compilé avant de pouvoir être utilisé. C'est aussi le moment de préciser la fonction de coût, l'optimiseur ainsi que la métrique.

Compilez le modèle, à l'aide de [`model.compile()`](https://keras.io/models/sequential/#compile) avec les paramètres ci-dessous

*   loss = 'binary_crossentropy'
*   optimizer = 'adam'
*   metrics = ['accuracy']

In [0]:
model.compile(### VOTRE CODE ICI ###)

On peut visualiser notre modèle

In [0]:
plot_model(model, "model.png")
Image("model.png")

Enfin, si on veut avoir plus de détails sur le réseau, par exemple la dimension ainsi que le nombre de paramètres de chaque couche, on peut utiliser la fonction `model.summary()`

In [0]:
model.summary()

### Entraînement du réseau

Une fois le réseau compilé, l'entraînement est simple comme bonjour.

Entraînez le modèle sur `X_train_padded` et `y_train`, à l'aide de la fonction [`model.fit()`](https://keras.io/models/sequential/#fit), avec les paramètres suivants
*   batch_size = 128
*   epochs = 5
*   validation_split = 0.05

In [0]:
history = model.fit(### VOTRE CODE ICI ###)

Il est souvent très utile de tracer l'accuracy et la loss le long de l'entraînement. 

In [0]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [0]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Evaluation du réseau

Pour évaluer un modèle de machine learning, l'approche canonique est de le tester sur un jeu de données, en l'occurrence, `X_test_padded`, que l'on a pas touché (du tout !) pendant l'entraînement.

A vos de coder !

> Indice : utilisez [cette fonction](https://keras.io/models/sequential/#evaluate). Pensez à augmenter `batch_size` (par défaut c'est 1), 128 est une valeur souvent utilisée.

In [0]:
loss, accuracy = model.evaluate(### VOTRE CODE ICI ###)
print(f"loss: {loss}")
print(f"accuracy: {accuracy}")

## Testez avec vos exemples !

Tester sur le jeu de test, c'est amusant. C'est encore plus intéressant si on teste sur notre propre "revue" !

Cette fois-ci, on a codé pour vous :) Jouez avec la fonction `predict` suivante !

In [0]:
def predict(sentence):
  sequences = tokenizer.texts_to_sequences([sentence])
  sequences_padded = pad_sequences(sequences, maxlen=maxlen)
  y_pred = model.predict(sequences_padded)
  y_pred = np.squeeze(y_pred)
  return "positive" if y_pred >= 0.5 else "negative"

In [0]:
predict("I really loved this film, it was fantastic!")

In [0]:
predict("I don't understand how anyone can be optimistic about this film.")